# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Data-Integration" data-toc-modified-id="Data-Integration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Integration</a></div><div class="lev2 toc-item"><a href="#Concatenating-(Stacking)-DataFrames-with-Index-Alignment" data-toc-modified-id="Concatenating-(Stacking)-DataFrames-with-Index-Alignment-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Concatenating (Stacking) DataFrames with Index Alignment</a></div><div class="lev2 toc-item"><a href="#The-Employee-Example-DB" data-toc-modified-id="The-Employee-Example-DB-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>The Employee Example DB</a></div><div class="lev2 toc-item"><a href="#Joining-DataFrames" data-toc-modified-id="Joining-DataFrames-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Joining DataFrames</a></div><div class="lev2 toc-item"><a href="#Checkpoint" data-toc-modified-id="Checkpoint-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Checkpoint</a></div>

# Data Integration (`.concat()`, `.merge()`)

In a wide range of situations, a data scientist will find data of interest is found in separate, but related datasets. Usually, for more effective data analysis and modelling (particularly in machine learning), one must merge these into a single, combined dataset. 

Take the example of analysis of climate variation and change in a certain geographical region of the planet. The necessary data weather measurements (e.g., daily temperatures) might have been collected separately by weather stations placed across the region. In order to gain broader insights about the data, those individual datasets must be combined.

Another scenario assumes that there are separate datasets for students and units, and one needs to query information that comes from both (e.g., unit's number of credits and student name). Typically, keys (e.g., `student_id`) are relied upon in the integration procedure.

It has beem claimed said (original source?) that about 80% of a data scientist's time and effort goes into data wrangling, and integration is part of that. All sorts of challenges one might face here, such as datasets with unmatching and inconsistent schemas, data types, ranges, and units of measurement, to mention a few. 

We have already seen `.concat` and index alignment for combining datasets. We then now explore `.merge` for joining datasets that are related by _keys_ (similar to working with relational databases and `JOIN`). A new example database `employee_db` with a set of related datasets is used.

The usual preamble for importing the essential modules and configuring the plotting engine.

In [2]:
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize': (8,6)}, style='ticks', context='notebook')

## The Employee Example DB

The zip file containing the set of `.csv` files can be obtained from. As usual, decompress it and place the relevant datasets into a suitable location (e.g., your OneDrive area).

Briefly, we have `employees` that work for `departments` based in certain `locations` that exist in `countries` which are placed in `regions`. The full set of relationships is as follows:

* `employees(department_id) -> departments(department_id)`
* `employees(manager_id) -> employees(employee_id)` (the manager is another employee)
* `departments(manager_id) -> employes(employee_id)`
* `departments(location_id) -> location(location_id)`
* `location(country_id) -> countries(country_id)`
* `countries(region_id) -> regions(region_id)`

In [3]:
employees_df = pd.read_csv('./data/employees.csv')
employees_df.head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
0,100,Steven,King,SKING,515.123.4567,1987-06-17,AD_PRES,24000.0,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,1989-09-21,AD_VP,17000.0,NaN,100.0,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,1993-01-13,AD_VP,17000.0,NaN,100.0,90.0
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,1990-01-03,IT_PROG,9000.0,NaN,102.0,60.0
4,104,Bruce,Ernst,BERNST,590.423.4568,1991-05-21,IT_PROG,6000.0,NaN,103.0,60.0


In [4]:
departments_df = pd.read_csv('./data/departments.csv')
departments_df.head(10)

,department_id,department_name,manager_id,location_id
0,10,Administration,200.0,1700
1,20,Marketing,201.0,1800
2,30,Purchasing,114.0,1700
3,40,Human Resources,203.0,2400
4,50,Shipping,121.0,1500
5,60,IT,103.0,1400
6,70,Public Relations,204.0,2700
7,80,Sales,145.0,2500
8,90,Executive,100.0,1700
9,100,Finance,108.0,1700


In [5]:
locations_df = pd.read_csv('./data/locations.csv')
locations_df.head()

,location_id,street_address,postal_code,city,state_province,country_id
0,1000,1297 Via Cola di Rie,00989,Roma,NaN,IT
1,1100,93091 Calle della Testa,10934,Venice,NaN,IT
2,1200,2017 Shinjuku-ku,1689,Tokyo,Tokyo Prefecture,JP
3,1300,9450 Kamiya-cho,6823,Hiroshima,NaN,JP
4,1400,2014 Jabberwocky Rd,26192,Southlake,Texas,US


In [6]:
countries_df = pd.read_csv('./data/countries.csv')
countries_df.head()

,country_id,country_name,region_id
0,IT,Italy,1
1,JP,Japan,3
2,US,United States of America,2
3,CA,Canada,2
4,CN,China,3


In [7]:
regions_df = pd.read_csv('./data/regions.csv')
regions_df.head()

,region_id,region_name
0,1,Europe
1,2,Americas
2,3,Asia
3,4,Middle East and Africa


## Joining DataFrames

In many situations, following good practices of _relational database design_ to, for example, prevent issues with data replication and update anomalies, data is spread across different datasets that are related by __keys__. In relational database's terminology, one dataset's __foreign key__ links to a dataset's __primary key__ (the latter, a unique identifier for each) instance of data.

For example, `employees(department_id)` is a foreign key to `departments(department_id)`. Those columns and their matching values are used to keep records (e.g., rows) from both datasets related.

In such cases, more often than not, _interesting questions_ about the data require __joining datasets__. With `pandas`, a natural way of achieving this is via the method `.merge`.

In [8]:
countries_df.head()

,country_id,country_name,region_id
0,IT,Italy,1
1,JP,Japan,3
2,US,United States of America,2
3,CA,Canada,2
4,CN,China,3


In [9]:
regions_df.head()

,region_id,region_name
0,1,Europe
1,2,Americas
2,3,Asia
3,4,Middle East and Africa


In [10]:
countries_df.merge(regions_df).sample(5, random_state=42)

,country_id,country_name,region_id,region_name
8,JP,Japan,3,Asia
16,BR,Brazil,2,Americas
0,IT,Italy,1,Europe
23,IL,Israel,4,Middle East and Africa
11,AU,Australia,3,Asia


It is always good practice to check the columns of each DataFrame to verify which columns are the keys, and how they are named.

In [11]:
employees_df.columns

Index(['employee_id', 'first_name', 'last_name', 'email', 'phone_number',
       'hire_date', 'job_id', 'salary', 'commission_pct', 'manager_id',
       'department_id'],
      dtype='object')

In [12]:
departments_df.columns

Index(['department_id', 'department_name', 'manager_id', 'location_id'], dtype='object')

In [15]:
# returns the intersection of two sets 
# (in this case, the columns the two DataFrames have in common)
departments_df.columns.intersection(employees_df.columns)

Index(['department_id', 'manager_id'], dtype='object')

The following is an example of joining two DataFrames on a specified column they have in common; rows from the two DataFrames are concatenated based on matching values of their joining column(s) (key(s)):

In [16]:
employees_df.merge(departments_df, on='department_id').head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id_x,department_id,department_name,manager_id_y,location_id
0,100,Steven,King,SKING,515.123.4567,1987-06-17,AD_PRES,24000.0,NaN,NaN,90.0,Executive,100.0,1700
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,1989-09-21,AD_VP,17000.0,NaN,100.0,90.0,Executive,100.0,1700
2,102,Lex,De Haan,LDEHAAN,515.123.4569,1993-01-13,AD_VP,17000.0,NaN,100.0,90.0,Executive,100.0,1700
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,1990-01-03,IT_PROG,9000.0,NaN,102.0,60.0,IT,103.0,1400
4,104,Bruce,Ernst,BERNST,590.423.4568,1991-05-21,IT_PROG,6000.0,NaN,103.0,60.0,IT,103.0,1400


For reducing clutter and help the illustration, I'll drop some employee features for the time being.

In [17]:
employees_sub_df = employees_df[
    ['employee_id', 'first_name', 'last_name', 'salary', 'department_id']
]
employees_sub_df.head()

,employee_id,first_name,last_name,salary,department_id
0,100,Steven,King,24000.0,90.0
1,101,Neena,Kochhar,17000.0,90.0
2,102,Lex,De Haan,17000.0,90.0
3,103,Alexander,Hunold,9000.0,60.0
4,104,Bruce,Ernst,6000.0,60.0


In [18]:
employees_sub_df.merge(departments_df, on='department_id').head()

,employee_id,first_name,last_name,salary,department_id,department_name,manager_id,location_id
0,100,Steven,King,24000.0,90.0,Executive,100.0,1700
1,101,Neena,Kochhar,17000.0,90.0,Executive,100.0,1700
2,102,Lex,De Haan,17000.0,90.0,Executive,100.0,1700
3,103,Alexander,Hunold,9000.0,60.0,IT,103.0,1400
4,104,Bruce,Ernst,6000.0,60.0,IT,103.0,1400


What if names of columns do not match?

In [19]:
departments_id_df = departments_df.rename(columns={'department_id': 'id'})
departments_id_df.head()

,id,department_name,manager_id,location_id
0,10,Administration,200.0,1700
1,20,Marketing,201.0,1800
2,30,Purchasing,114.0,1700
3,40,Human Resources,203.0,2400
4,50,Shipping,121.0,1500


...then, use the options `left_on=` and `right_on=`.

In [20]:
employees_sub_df.merge(departments_id_df, left_on='department_id', right_on='id').head()

,employee_id,first_name,last_name,salary,department_id,id,department_name,manager_id,location_id
0,100,Steven,King,24000.0,90.0,90,Executive,100.0,1700
1,101,Neena,Kochhar,17000.0,90.0,90,Executive,100.0,1700
2,102,Lex,De Haan,17000.0,90.0,90,Executive,100.0,1700
3,103,Alexander,Hunold,9000.0,60.0,60,IT,103.0,1400
4,104,Bruce,Ernst,6000.0,60.0,60,IT,103.0,1400


And if one of the matching parts is actually the row index?

In [21]:
departments_i_df = departments_df.set_index('department_id')
departments_i_df.head()

,department_name,manager_id,location_id
department_id,,,
10,Administration,200.0,1700
20,Marketing,201.0,1800
30,Purchasing,114.0,1700
40,Human Resources,203.0,2400
50,Shipping,121.0,1500


In [22]:
employees_sub_df.merge(departments_i_df, left_on='department_id', right_index=True).head()

,employee_id,first_name,last_name,salary,department_id,department_name,manager_id,location_id
0,100,Steven,King,24000.0,90.0,Executive,100.0,1700
1,101,Neena,Kochhar,17000.0,90.0,Executive,100.0,1700
2,102,Lex,De Haan,17000.0,90.0,Executive,100.0,1700
3,103,Alexander,Hunold,9000.0,60.0,IT,103.0,1400
4,104,Bruce,Ernst,6000.0,60.0,IT,103.0,1400


In [23]:
departments_i_df.merge(employees_sub_df, left_index=True, right_on='department_id').head()

,department_name,manager_id,location_id,employee_id,first_name,last_name,salary,department_id
100,Administration,200.0,1700,200,Jennifer,Whalen,4400.0,10.0
101,Marketing,201.0,1800,201,Michael,Hartstein,13000.0,20.0
102,Marketing,201.0,1800,202,Pat,Fay,6000.0,20.0
14,Purchasing,114.0,1700,114,Den,Raphaely,11000.0,30.0
15,Purchasing,114.0,1700,115,Alexander,Khoo,3100.0,30.0


It is worth pointing out that `.merge` takes **two DataFrames as input** and produces a **new DataFrame as output**. As with other operations on DataFrames, one could **chain** `.merge` calls for joining multiple datasets. For example, what if we would to obtain a DataFrame containing the **name of the department** and the **country** in which they are based?

In [24]:
(departments_df
    .merge(locations_df, on='location_id')
    .merge(countries_df, on='country_id')
    [['department_name', 'country_name']]
).sample(5, random_state=42)

,department_name,country_name
8,Shareholder Services,United States of America
13,Operations,United States of America
9,Benefits,United States of America
21,Shipping,United States of America
0,Administration,United States of America


## Checkpoint

Produce a DataFrame that shows the departments together with the addresses of their locations.

Produce a DataFrame containing the name of the department and the cities in which they are based.

Produce a DataFrame with the name of the department and the name of the country in which they are based.

Produce a DataFrame with the name of the department and the name of the region of the world in which they are based.


Produce a DataFrame with the top-5 earners in the IT and Sales department.

Produce a DataFrame with the top-10 earners in United Kingdom. In the output, include only the employee's name and salary, and the name of the deparment they work for.

How many employess work in Canada?